## Install Dependencies

In [1]:
!pip install aixplain --upgrade 

  Attempting uninstall: aixplain
    Found existing installation: aiXplain 0.2.20
    Uninstalling aiXplain-0.2.20:
      Successfully uninstalled aiXplain-0.2.20


## SEARCH TOOL

In [2]:
%env TEAM_API_KEY=

env: TEAM_API_KEY=


In [3]:
!env | grep TEAM_API_KEY

TEAM_API_KEY=


In [ ]:
def main(query: str):
  import requests, json
  PROMPT = """
    Given the parsed page of an online product. Try to get the following information about the product, name, description, price, rate.
    Return your response in a json format like the following:
    {
      "name": <name>,
      "url": <url>,
      "description": <description>,
      "price": <price>,
      "rate": <rate>
    }
    Only return the json output in a format to be parsed directly using json.loads.
    The url is supposed to be the first line in the inout given to you.
    If the input parsed page doesn't seem to be of an online product, return {}
    PARSED PAGE:
  """
  search_model_url = "https://models.aixplain.com/api/v2/execute/65c51c556eb563350f6e1bb1"
  scrapping_model_url = "https://models.aixplain.com/api/v2/execute/66f423426eb563fa213a3531"
  llm_model_url = "https://models.aixplain.com/api/v2/execute/677c16166eb563bb611623c1"
  api_key = "YOUR_API_KEY"
  headers = {"x-api-key": api_key, "Content-Type": "application/json"}

  payload = {
      "text": f"I want to buy {query} online",
      "numResults": 100
  }
  r = requests.post(search_model_url, headers=headers, json=payload)
  resp = r.json()
  final_output = []
  for item in resp['details']:
    url = item['document']
    scrapping_payload = {
        "text": url
    }
    try:
      r = requests.post(scrapping_model_url, headers=headers, json=scrapping_payload)
      resp = r.json()
      llm_payload = {
          "text" : PROMPT + "\n" + url + "\n" + resp['data']
      }
      r = requests.post(llm_model_url, headers=headers, json=llm_payload)
      resp = r.json()
      try:
        json_output = json.loads(resp['data'].replace("```", ""))
        if json_output:
          final_output.append(json_output)
      except:
        continue

    except:
      continue




  return final_output

#main("iphone 16")


In [ ]:
from aixplain.factories import ModelFactory
utility_search_model = ModelFactory.create_utility_model(
    name="Google Search",
    code=main,
    description="Search Google for the query"
    )

In [ ]:
from aixplain.factories import AgentFactory
online_shopping_assistant = AgentFactory.create(
    name="Online Shopping Assistant4",
    tools=[utility_search_model],

    llm_id="677c16166eb563bb611623c1",
    description="An Agent for to help with online shopping.",
    instructions="""
      Given a user query, help the user decide where to buy it with the best price and rate.
      Start your response with "Here are the results I got:"
      Give the user as much information as possible that will help decide what to buy like the cheapest and the top rated product.
      Present your results in the following format:
          '''
          Product Name: <name>
          Description: <description>
          Price: <price>
          Rate: <rate>
          Url: <url>
          -------------------------------------
          Product Name: <name>
          Description: <description>
          Price: <price>
          Rate: <rate>
          Url: <url>
          -------------------------------------
          Product Name: <name>
          Description: <description>
          Price: <price>
          Rate: <rate>
          Url: <url>
          -------------------------------------
          Product Name: <name>
          Description: <description>
          Price: <price>
          Rate: <rate>
          Url: <url>
          '''
      You should give the user as much information as possible that will help decide what to buy.
    """
)
online_shopping_assistant.id

/usr/local/lib/python3.11/dist-packages/aixplain/factories/agent_factory/__init__.py:78: UserWarning: The 'instructions' parameter was recently added and serves the same purpose as 'description' did previously: set the role of the agent as a system prompt. The 'description' parameter is still required and should be used to set a short summary of the agent's purpose. For the next releases, the 'instructions' parameter will be required.
  warnings.warn(


'67bf2249058286b62912e2dc'

In [ ]:
result = online_shopping_assistant.run("whiteboard")

In [ ]:
print(result['data']['output'])

"Here are the results I got: 
          Product Name: Quartet Whiteboard
          Description: A high-quality whiteboard for home or office use
          Price: $15
          Rate: 4.5
          Url: https://www.amazon.com/Quartet-Whiteboard-6x4-Feet-Aluminum-Frame
          -------------------------------------
          Product Name: Viz-Pro Whiteboard
          Description: A durable and easy-to-clean whiteboard for heavy use
          Price: $25
          Rate: 4.8
          Url: https://www.staples.com/Viz-Pro-Whiteboard-6x4-Feet-Aluminum-Frame
          -------------------------------------
          Product Name: Lockways Whiteboard
          Description: A budget-friendly whiteboard for basic use
          Price: $10
          Rate: 4.2
          Url: https://www.walmart.com/Lockways-Whiteboard-6x4-Feet-Plastic-Frame
          -------------------------------------
          Product Name: Ghent Whiteboard
          Description: A high-end whiteboard with a sleek and modern desi

In [ ]:
online_shopping_assistant.deploy()